# Getting Started with Snowflake Semantic View

This notebook guides you through setting up and querying a Snowflake Semantic View using TPC-DS sample data. You will learn how to:

1.  Create a new database and schema.
2.  Create views from existing sample data tables.
3.  Define a Semantic View to simplify data analysis.
4.  Query the Semantic View.
5.  Explore the Semantic View in Cortex Analyst.

Let's get started!

## Step 1: Verify your Environment Setup

Before proceeding, let's ensure our warehouse, database, and schema are correctly set, and then list the views we just created.

In [ ]:
-- Select the warehouse, database, and schema
USE DATABASE <mdt3_default_database##>;          --change this
USE SCHEMA default_schema;              

-- Show all views in the current schema to verify creation
SHOW VIEWS;

## Step 2: Define the Semantic View

Now, we'll define our `TPCDS_SEMANTIC_VIEW_SM` semantic view. This view will establish relationships between our tables, define facts (measures), and dimensions (attributes), making it easier to query and analyze our data without complex joins.

In [ ]:
use role accountadmin;

-- Create or replace the semantic view named TPCDS_SEMANTIC_VIEW_SM
CREATE OR REPLACE SEMANTIC VIEW TPCDS_SEMANTIC_VIEW_SM
	tables (
		CUSTOMER PRIMARY KEY (C_CUSTOMER_SK)
                 with synonyms('BUYER', 'PURCHASER', 'CLIENT', 'CONSUMER')
                 comment = 'This table stores customer demographic information, specifically the birth year and country of origin, uniquely identified by a customer identifier (C_CUSTOMER_SK).',
		DATE AS DATE_DIM PRIMARY KEY (D_DATE_SK) 
                 with synonyms('calendar', 'date_reference')
                 comment = 'This table stores a dimension of dates, providing a single row for each date, with corresponding surrogate key, month of year, week sequence, and year, to support date-based analysis and reporting in a data warehouse or business intelligence environment',
		DEMO AS CUSTOMER_DEMOGRAPHICS PRIMARY KEY (CD_DEMO_SK)
                 with synonyms('client_data', 'demographic_data')
                 comment = 'This table stores demographic information about customers, including a unique demographic key, credit rating, and marital status.',
		ITEM PRIMARY KEY (I_ITEM_SK)
                 with synonyms('product', 'merchandise', 'goods', 'commodity')
                 comment = 'This table stores information about individual items in an inventory, including the brand, category, and class of the item, as well as its unique identifier (I_ITEM_SK), wholesale cost, and current selling price.',
		STORE PRIMARY KEY (S_STORE_SK)
                 with synonyms('retail_store', 'shop', 'retail_outlet', 'market_store')
                 comment = 'This table stores information about retail stores, including their market identification, floor space, location (state and country), unique store identifier, and tax percentage applicable to the store.',
		STORESALES as STORE_SALES PRIMARY KEY (SS_STORE_SK)
                 with synonyms('store_sales', 'sales_data', 'store_revenue')
                 comment = 'This table captures sales data for a retail store, including demographic information about the customer, the item sold, the date of sale, the store location, the sales price, and the quantity sold.'
	)
	relationships (
		SALESTOCUSTOMER as STORESALES(SS_CUSTOMER_SK) references CUSTOMER(C_CUSTOMER_SK),
		SALESTODATE as STORESALES(SS_SOLD_DATE_SK) references DATE(D_DATE_SK),
		SALESTODEMO as STORESALES(SS_CDEMO_SK) references DEMO(CD_DEMO_SK),
		SALESTOITEM as STORESALES(SS_ITEM_SK) references ITEM(I_ITEM_SK),
		SALETOSTORE as STORESALES(SS_STORE_SK) references STORE(S_STORE_SK)
	)
	facts (
		ITEM.COST as I_WHOLESALE_COST
                 with synonyms('price', 'bulk_cost', 'item_cost')
                 comment = 'The wholesale cost of an item, representing the price at which the item is purchased from the supplier.',
		ITEM.PRICE as i_current_price
                 with synonyms('current_cost', 'current_value', 'item_price')
                 comment = 'The current price of an item.',
		STORE.TAX_RATE as S_TAX_PRECENTAGE
                 with synonyms('tax_rate', 'tax_percent', 'vat_rate')
                 comment = 'The percentage of sales tax applied to transactions at each store location.',
        STORESALES.SALES_QUANTITY as SS_QUANTITY
                 with synonyms('item_count', 'quantity_sold', 'product_quantity')
                 comment = 'The quantity of items sold in a single transaction at the store.'
	)
    dimensions (
		CUSTOMER.BIRTHYEAR as C_BIRTH_YEAR
                 with synonyms('birth_year', 'year_of_birth')
                 comment = 'The year in which the customer was born',
		CUSTOMER.COUNTRY as C_BIRTH_COUNTRY
                 with synonyms('nationality', 'native_country')
                 comment = 'The country where the customer was born',
		CUSTOMER.C_CUSTOMER_SK as c_customer_sk
                 with synonyms('customer_id', 'customer_key', 'customer_number')
                 comment = 'Unique identifier for a customer in the customer dimension',
		DATE.DATE as D_DATE
                 with synonyms('calendar_date','date_value')
                 comment = 'Date dimension representing individual dates, allowing for analysis and filtering by specific days.',
		DATE.D_DATE_SK as d_date_sk
                 with synonyms('date_key', 'date_surrogate_key', 'date_id')
                 comment = 'Unique identifier for a specific date in the date dimension, used to join with fact tables.',
		DATE.MONTH as D_MOY
                 with synonyms('month_of_year', 'month_number', 'month_value', 'month')
                 comment = 'Month of the year, represented as a numerical value (1-12), where 1 corresponds to January and 12 corresponds to December.',
		DATE.WEEK as D_WEEK_SEQ
                 with synonyms('week_sequence', 'week_number')
                 comment = 'A sequential identifier for the week within a given year, starting from 1 for the first week and incrementing by 1 for each subsequent week.',
		DATE.YEAR as D_YEAR
                 with synonyms('calendar_year', 'year_number')
                 comment = 'The year in which a date falls, represented as a four-digit integer.',
		DEMO.CD_DEMO_SK as cd_demo_sk
                 with synonyms('demographic_key', 'demographic_id')
                 comment = 'Unique identifer for the customer demographic_record',
		DEMO.CREDIT_RATING as CD_CREDIT_RATING
                 with synonyms('credit_score', 'credit_rank', 'credit_status')
                 comment = 'A categorical dimension indicating the creditworthiness of a customer, with values representing a good credit rating, low risk of default, or high risk of default.',
		DEMO.MARITAL_STATUS as CD_MARITAL_STATUS
                 with synonyms('relationship_status', 'marriage_status')
                 comment = 'The marital status of the customer, with possible values being Married (M), Single (S), or Divorced (D)',
		ITEM.BRAND as I_BRAND
                 with synonyms('brand_name', 'item_brand', 'product_brand')
                 comment = 'The brand or manufacturer of the item.',
		ITEM.CATEGORY as I_CATEGORY
                 with synonyms('product_category', 'category_name', 'product_group')
                 comment = 'Category of the item, which can be Sports, Men, or Music, indicating the type of product or merchandise.',
		ITEM.CLASS as I_CLASS
                 with synonyms('item_class', 'product_category', 'item_type')
                 comment = 'The type of item being sold, such as a baseball-themed item, a classical-style item, or a womens watch.',
		ITEM.I_ITEM_SK as i_item_sk
                 with synonyms('item_identifier')
                 comment = 'Unique identifier for an item in the inventory.',
		STORE.MARKET as S_MARKET_ID
                 with synonyms('market_identifier', 'market_code')
                 comment = 'Unique identifier for a specific store market.',
		STORE.SQUAREFOOTAGE as S_FLOOR_SPACE
                 with synonyms('square_footage', 'floor_area', 'retail_space')
                 comment = 'The total square footage of floor space in the store.',
		STORE.STATE as S_STATE
                 with synonyms('state_code', 'location')
                 comment = 'State where the store is located.',
		STORE.STORECOUNTRY as S_COUNTRY
                 with synonyms('nation', 'land', 'territory', 'state', 'region')
                 comment = 'The country where the store is physically located.',
		STORE.S_STORE_SK as s_store_sk
                 with synonyms('store_key', 'store_id', 'store_number', 'store_identifier')
                 comment = 'Unique identifier for a store.',
		STORESALES.SS_CDEMO_SK as ss_cdemo_sk
                 with synonyms('customer_demo_identifier', 'demographic_identifier')
                 comment = 'Customer demographic key.',
		STORESALES.SS_CUSTOMER_SK as ss_customer_sk
                 with synonyms('customer_id', 'customer_identifier')
                 comment = 'Unique identifier for the customer who made the sale.',
		STORESALES.SS_ITEM_SK as ss_item_sk
                 with synonyms('item_id', 'item_number')
                 comment = 'Unique identifier for the item sold.',
		STORESALES.SS_SOLD_DATE_SK as ss_sold_date_sk
                 with synonyms('sale_date', 'transaction_date')
                 comment = 'Date key for the date the sale was made, in the format of a unique integer value representing the date in the format YYYYMMDD.',
		STORESALES.SS_STORE_SK as ss_store_sk
                 with synonyms('store_key', 'store_id', 'store_number')
                 comment = 'Unique identifier for the store where the sales transaction occurred.'
)
	metrics (
		STORESALES.TOTALCOST as SUM(item.cost)
                 with synonyms('total_cost')
                 comment = 'The total cost of an item, representing the price at which the item is purchased from the supplier.',
		STORESALES.TOTALSALESPRICE as SUM(SS_SALES_PRICE)
                 with synonyms('price')
                 comment = 'The total amount of money earned from sales of all items in a store.',
		STORESALES.TOTALSALESQUANTITY as SUM(SS_QUANTITY)
                 WITH SYNONYMS = ( 'total sales quantity', 'total sales amount')
                 comment = 'The total quantity of items sold in a single transaction.'
)
;

## Step 3: Verify the Semantic View Creation

Let's confirm that our semantic view has been successfully created by listing all semantic views in the current database.

In [ ]:
-- Lists semantic views in the database that is currently in use
SHOW SEMANTIC VIEWS;

## Step 4: Describe the Semantic View

To understand the structure and components of our newly created semantic view, we can use the `DESC SEMANTIC VIEW` command. This will provide details about its tables, relationships, facts, and dimensions.

In [ ]:
-- Describes the semantic view named TPCDS_SEMANTIC_VIEW_SM, and as a special bonus uses our new flow operator to filter and project only the metric and dimension names
DESC SEMANTIC VIEW TPCDS_SEMANTIC_VIEW_SM
    ->> SELECT "object_kind","property_value" as "parent_object","object_name" FROM $1
        WHERE "object_kind" IN ('METRIC','DIMENSION') AND "property" IN ('TABLE')
;

## Step 5: "Talk To" the Semantic View with Cortex Analyst

Snowflake's Cortex Analyst allows you to interact with your semantic views using natural language. 

Let's dynamically generate a link to Cortex Analyst so that you can access the semantic view.

Go to the link in the cell below:

In [ ]:
SELECT 'https://app.snowflake.com/' || CURRENT_ORGANIZATION_NAME() || '/' || CURRENT_ACCOUNT_NAME() || '/#/studio/analyst/databases/SAMPLE_DATA/schemas/TPCDS_SF10TCL/semanticView/TPCDS_SEMANTIC_VIEW_SM/edit' AS RESULT;

In [ ]:
import streamlit as st

link = sql_step7.to_pandas()['RESULT'].iloc[0]

st.link_button("Go to Cortex Analyst", link)

You can ask in natural language:

*Show me the top selling brands in by total sales quantity in the state 'TX' in the 'Books' category in the year 2003*

## Step 6: Query the Semantic View Using SQL

Now that our semantic view is defined, we can easily query it to retrieve aggregated data. The following query demonstrates how to find the top-selling brands in a specific state and category for a given year and month, leveraging the simplified structure provided by the semantic view.

In [ ]:
-- Query the semantic view to find top selling brands
SELECT * FROM SEMANTIC_VIEW
( 
 TPCDS_SEMANTIC_VIEW_SM
    DIMENSIONS 
            Item.Brand,
            Item.Category,            
            Date.Year,
            Date.Month,
            Store.State
    METRICS 
        StoreSales.TotalSalesQuantity
    WHERE
        Date.Year = '2002' AND Date.Month = '12' AND Store.State ='TX' AND Item.Category = 'Books'
) 
ORDER BY TotalSalesQuantity DESC LIMIT 10;

## Step 7 (Optional): Build an Interactive Data App

In this step, we'll build 2 simple interactive data apps:

1. Interactive data visualization app
2. Simple interactive dashboard

Firstly, we'll modify the SQL query to show data for month 12.

In [ ]:
-- Query the semantic view for month 12
SELECT * FROM SEMANTIC_VIEW
( 
 TPCDS_SEMANTIC_VIEW_SM
    DIMENSIONS 
            Item.Brand,
            Item.Category,            
            Date.Year,
            Date.Month,
            Store.State
    METRICS 
        StoreSales.TotalSalesQuantity
    WHERE
        Date.Year = '2002' AND Date.Month = '12' AND Item.Category = 'Books'
) 
ORDER BY TotalSalesQuantity DESC;

Next, we'll convert the SQL table to a Pandas DataFrame.

In [ ]:
cell1.to_pandas()

### App 1. Interactive Data Visualization

Here the user can interactively explore the sales data:

In [ ]:
import streamlit as st
import pandas as pd

st.title("📊 Sales Data Interactive Visualization")

# Create selectbox for grouping option
group_by = st.selectbox(
    "Select grouping option:",
    options=['BRAND', 'STATE'],
    index=0
)

# Group the data based on selection
if group_by == 'BRAND':
    grouped_data = df.groupby('BRAND')['TOTALSALESQUANTITY'].sum().reset_index()
    grouped_data = grouped_data.set_index('BRAND')
    chart_title = "Total Sales Quantity by Brand"
else:  # group_by == 'STATE'
    grouped_data = df.groupby('STATE')['TOTALSALESQUANTITY'].sum().reset_index()
    grouped_data = grouped_data.set_index('STATE')
    chart_title = "Total Sales Quantity by State"

# Display the chart
st.subheader(chart_title)
st.bar_chart(grouped_data['TOTALSALESQUANTITY'])

# Optional: Display the data table
if st.checkbox("Show data table"):
    st.subheader("Grouped Data")
    st.dataframe(grouped_data)

### App 2. Dashboard

Here's a simple dashboard we're we've included a row of metrics:

In [ ]:
import streamlit as st
import pandas as pd

st.title("📊 Sales Data Dashboard")

# Create selectbox for grouping option
group_by = st.selectbox(
    "Select grouping option:",
    options=['BRAND', 'STATE'],
    index=0
)

# Group the data based on selection
if group_by == 'BRAND':
    grouped_data = df.groupby('BRAND')['TOTALSALESQUANTITY'].sum().reset_index()
    grouped_data = grouped_data.set_index('BRAND')
    chart_title = "Total Sales Quantity by Brand"
else:  # group_by == 'STATE'
    grouped_data = df.groupby('STATE')['TOTALSALESQUANTITY'].sum().reset_index()
    grouped_data = grouped_data.set_index('STATE')
    chart_title = "Total Sales Quantity by State"

# Calculate KPIs based on current grouping
total_sales = df['TOTALSALESQUANTITY'].sum()
avg_sales = df['TOTALSALESQUANTITY'].mean()
top_performer = grouped_data['TOTALSALESQUANTITY'].max()
top_performer_name = grouped_data['TOTALSALESQUANTITY'].idxmax()

# Display KPI metrics in 3 columns
col1, col2, col3 = st.columns(3)

with col1:
    st.metric(
        label="Total Sales Quantity", 
        value=f"{total_sales:,.0f}",
        delta=None
    )

with col2:
    if group_by == 'BRAND':
        st.metric(
            label="Average Sales per Brand", 
            value=f"{avg_sales:,.0f}",
            delta=f"{((avg_sales/total_sales)*100):.3f}% of total"
        )
    else:
        st.metric(
            label="Average Sales per State", 
            value=f"{avg_sales:,.0f}",
            delta=f"{len(df['STATE'].unique())} state(s)"
        )

with col3:
    st.metric(
        label=f"Top {group_by.title()}", 
        value=f"{top_performer:,.0f}",
        delta=f"{top_performer_name}"
    )

# Display the chart
st.subheader(chart_title)
st.bar_chart(grouped_data['TOTALSALESQUANTITY'])

# Optional: Display the data table
if st.checkbox("Show data table"):
    st.subheader("Grouped Data")
    st.dataframe(grouped_data)

## Related Resources

Articles:

- [Using SQL commands to create and manage semantic views](https://docs.snowflake.com/user-guide/views-semantic/sql)
- [Using the Cortex Analyst Semantic View Generator](https://docs.snowflake.com/en/user-guide/views-semantic/ui)
- [Sample Data: TPC-DS](https://docs.snowflake.com/en/user-guide/sample-data-tpcds)
- [TPC-DS Benchmark Overview](https://www.tpc.org/tpcds/) - Understanding the sample dataset used in this guide

Documentation:
- [Overview of semantic views](https://docs.snowflake.com/en/user-guide/views-semantic/overview)
- [CREATE SEMANTIC VIEW](https://docs.snowflake.com/en/sql-reference/sql/create-semantic-view)
- [DROP SEMANTIC VIEW](https://docs.snowflake.com/en/sql-reference/sql/drop-semantic-view)
- [SHOW SEMANTIC VIEWS](https://docs.snowflake.com/en/sql-reference/sql/show-semantic-views)
- [DESCRIBE SEMANTIC VIEW](https://docs.snowflake.com/en/sql-reference/sql/desc-semantic-view)